In [115]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import glob
# import pandas as pd
from torch.utils.data import DataLoader

In [116]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

data = torchvision.datasets.ImageFolder(
                    root='./data/trainset/',
                    transform=transforms
                    )

test_data = torchvision.datasets.ImageFolder(
                    root= './data/test/', 
                    transform=transforms)

dataset_ratio = np.array([95, 5])/100

sizes = [int(x*len(data)) for x in dataset_ratio]
sizes[0] += len(data) - sum(sizes)

train_dataset, valid_dataset = torch.utils.data.random_split(dataset=data, lengths=sizes)

train_loader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

valid_loader = torch.utils.data.DataLoader(
                    valid_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

test_loader = torch.utils.data.DataLoader(
                    test_data,
                    batch_size=64,
                    num_workers=2,
                    shuffle=False
                    )
print(data)
print(valid_dataset)

In [3]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


## Hyper-parameters of the model

In [ ]:
learning_rate = 1e-3
lr0 = 0.002
optim = 'adam'
num_epochs = 10
store_every = 200

## Defining a residual convolution block to be used in the models

In [5]:
class ResidualConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3, 3), padding=1, activation = nn.ReLU):
        super(ResidualConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.activation = activation
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size)
        self.relu1 = nn.ReLU(in_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size)
        self.relu2 = nn.ReLU(in_channels)
        
        if in_channels != out_channels:
#             self.projected_conv = self.conv1x1(in_channels, out_channels)
            self.project_linear = nn.Linear(in_channels, out_channels)
    def forward(x):
        identity = x

        out = self.conv1(x)
        out = self.relu1(out)

        out = self.conv2(out)
        if self.in_channels == self.out_channels:
            out += identity
        else:
            out += self.project_linear(identity)
        out = self.relu2(out)
        
    def conv1x1(in_channels, out_channels, stride=1):
        """1x1 convolution"""
        print()
        return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

In [6]:
class ResidualClassifier(nn.Module):
    def __init__(self):
        super(ResidualClassifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            ResidualConv(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            ResidualConv(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            ResidualConv(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            )
        # Logistic Regression
        self.clf = nn.Linear(64, 2)
        
        def forward(x):
            return self.clf(self.conv(x).squeeze())

## Vanila CNN

In [117]:
class VanilaCNN(nn.Module):
    """Convnet Classifier"""
    def __init__(self):
        super(VanilaCNN, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 5
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 6
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(256, 2)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())

## First Implementation

In [111]:
print(model)

VanilaCNN(
  (conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1)

In [10]:
summary(model, input_size=(3,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             448
              ReLU-2           [-1, 16, 64, 64]               0
         MaxPool2d-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 32, 32, 32]           4,640
              ReLU-5           [-1, 32, 32, 32]               0
         MaxPool2d-6           [-1, 32, 16, 16]               0
            Conv2d-7           [-1, 64, 16, 16]          18,496
              ReLU-8           [-1, 64, 16, 16]               0
         MaxPool2d-9             [-1, 64, 8, 8]               0
           Conv2d-10            [-1, 128, 8, 8]          73,856
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Conv2d-13            [-1, 128, 4, 4]         147,584
             ReLU-14            [-1, 12

In [112]:
## Second implementation

In [113]:
clf = Classifier()
if cuda_available:
    clf = clf.cuda()
optimizer = torch.optim.SGD(clf.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [114]:
c = 0
acc = []
epoches= 50
for epoch in range(epoches):
    losses = []
    # Train
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """


    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Compute confusion matrix\
y_pred, y_test = trainer.predict()
class_names = ['Cat', 'Dog']
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

        outputs = clf(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        accuracy = 100. * (correct/total)
        acc.append(accuracy)

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print('--------------------------------------------------------------')
    clf.train()
    
    c += 1
     
    if (c%5 == 0)  and (acc[epoch] <= min(acc[epoch-4:epoch-1])):
        learning_rate *= 0.1

Epoch : 0 Loss : 0.950 
Epoch : 0 Loss : 0.704 
Epoch : 0 Loss : 0.659 
Epoch : 0 Loss : 0.637 
Epoch : 0 Loss : 0.616 
Epoch : 0 Test Acc : 71.000
--------------------------------------------------------------
Epoch : 1 Loss : 0.578 
Epoch : 1 Loss : 0.514 
Epoch : 1 Loss : 0.505 
Epoch : 1 Loss : 0.506 
Epoch : 1 Loss : 0.504 
Epoch : 1 Test Acc : 75.000
--------------------------------------------------------------
Epoch : 2 Loss : 0.435 
Epoch : 2 Loss : 0.426 
Epoch : 2 Loss : 0.430 
Epoch : 2 Loss : 0.432 
Epoch : 2 Loss : 0.434 
Epoch : 2 Test Acc : 75.000
--------------------------------------------------------------
Epoch : 3 Loss : 0.384 
Epoch : 3 Loss : 0.370 
Epoch : 3 Loss : 0.374 
Epoch : 3 Loss : 0.374 
Epoch : 3 Loss : 0.371 
Epoch : 3 Test Acc : 76.000
--------------------------------------------------------------
Epoch : 4 Loss : 0.353 
Epoch : 4 Loss : 0.327 
Epoch : 4 Loss : 0.327 
Epoch : 4 Loss : 0.333 
Epoch : 4 Loss : 0.338 
Epoch : 4 Test Acc : 75.000
--------

Epoch : 38 Loss : 0.001 
Epoch : 38 Loss : 0.001 
Epoch : 38 Loss : 0.001 
Epoch : 38 Loss : 0.001 
Epoch : 38 Test Acc : 80.000
--------------------------------------------------------------
Epoch : 39 Loss : 0.000 
Epoch : 39 Loss : 0.000 
Epoch : 39 Loss : 0.001 
Epoch : 39 Loss : 0.001 
Epoch : 39 Loss : 0.001 
Epoch : 39 Test Acc : 79.000
--------------------------------------------------------------
Epoch : 40 Loss : 0.000 
Epoch : 40 Loss : 0.002 
Epoch : 40 Loss : 0.002 
Epoch : 40 Loss : 0.002 
Epoch : 40 Loss : 0.002 
Epoch : 40 Test Acc : 81.000
--------------------------------------------------------------
Epoch : 41 Loss : 0.000 
Epoch : 41 Loss : 0.000 
Epoch : 41 Loss : 0.001 
Epoch : 41 Loss : 0.001 
Epoch : 41 Loss : 0.001 
Epoch : 41 Test Acc : 80.000
--------------------------------------------------------------
Epoch : 42 Loss : 0.000 
Epoch : 42 Loss : 0.003 
Epoch : 42 Loss : 0.002 
Epoch : 42 Loss : 0.002 
Epoch : 42 Loss : 0.002 
Epoch : 42 Test Acc : 80.000
---

In [ ]:
results = [[]]
for batch_idx, (inputs, targets) in enumerate(test_loader):
    if cuda_available:
        inputs, targets = inputs.cuda(), targets.cuda()
    outputs = clf(inputs)
    _, predicted = torch.max(outputs.data, 1)
    results = np.append(results, predicted.cpu().numpy())

results = np.int8(results)

## Creating submission csv file

In [ ]:
df = pd.DataFrame({ 'id': range(1, len(results)+1),
                    'label': results})
df['label'].replace([0,1], ['Cat','Dog'], inplace=True)
df[df.columns].to_csv('submisstion.csv',index=False)
print('Done...')

In [ ]:
def trainer1():
    for epoch in range(epoches):
        losses = []
        # Train
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            if cuda_available:
                inputs, targets = inputs.cuda(), targets.cuda()
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            losses.append(loss.data.item())


            if batch_idx%50==0:
                print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))

        # Evaluate
        model.eval()
        total = 0
        correct = 0
        for batch_idx, (inputs, targets) in enumerate(valid_loader):
            if cuda_available:
                inputs, targets = inputs.cuda(), targets.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
        print('--------------------------------------------------------------')
        model.train()
    print('Done...')

trainer1()